#Dependencies


In [4]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install ffmpeg
!pip install pydub
!pip install SpeechRecognition
!pip install translate
!pip install TTS
!pip install moviepy pydub SpeechRecognition pydub pyaudio


  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: Could not build wheels for pyaudio, which is required to install pyproject.toml-based projects


# Step 1 and Step 2: Extract and purify..
Video -> only Audio -> Only speech

In [11]:
import moviepy.editor as mp
from pydub import AudioSegment
!pip install noisereduce
import noisereduce as nr
import os

# Step 1: Input a video and convert it into audio
import subprocess

# Input video file path
input_video_path = "/content/sample_video.mp4"

# Output video file path
output_video_path = "only_video.mp4"

# Output audio file path
output_audio_path = "only_audio.wav"

# Run ffmpeg command to extract video
subprocess.run(["ffmpeg", "-i", input_video_path, "-c:v", "copy", output_video_path])

# Run ffmpeg command to extract audio
subprocess.run(["ffmpeg", "-i", input_video_path, "-vn", "-acodec", "pcm_s16le", "-ar", "44100", output_audio_path])



CompletedProcess(args=['ffmpeg', '-i', '/content/sample_video.mp4', '-vn', '-acodec', 'pcm_s16le', '-ar', '44100', 'only_audio.wav'], returncode=0)

In [14]:



# Step 2: Extract pure speech from the only_audio and save it as vocals
def remove_background_noise(audio_file):
    audio = AudioSegment.from_wav(audio_file)
    # Apply noise reduction (you might need to adjust parameters)
    reduced_noise = nr.reduce_noise(audio.get_array_of_samples(), audio.frame_rate)
    # Save the cleaned audio
    cleaned_audio = AudioSegment(
        data=reduced_noise.tobytes(),
        sample_width=audio.sample_width,
        frame_rate=audio.frame_rate,
        channels=audio.channels
    )
    cleaned_audio.export("vocals.wav", format="wav")
    return "vocals.wav"

if __name__ == "__main__":
    input_video_path = input("Enter the path of the video file: ")

    if not os.path.exists(input_video_path):
        print("File not found.")
    else:
        # Remove background noise from the extracted audio
        cleaned_audio_file = remove_background_noise("/content/only_audio.wav")
        output_audio_path="/content/only_audio.wav"
        print(f"Background noise removed. Cleaned audio saved as '{cleaned_audio_file}'")


#MOYE MOYE

Enter the path of the video file: /content/sample_video.mp4
Background noise removed. Cleaned audio saved as 'vocals.wav'


# Transcription & Translation
Vocals.mp3 -> Vocals.txt -> translated vocal_text.txt

In [15]:
from pydub import AudioSegment
import speech_recognition as sr
from translate import Translator
import numpy as np

def transcribe_and_translate_audio(audio_path, target_language='en'):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_wav(audio_path)

    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio_data)

        # Use the 'translate' library for translation
        translator = Translator(to_lang=target_language)
        translated_text = translator.translate(text)

        return translated_text
    except sr.UnknownValueError:
        print("Speech Recognition could not understand audio")
        return ""
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return ""

# Example usage:
vocals_text = transcribe_and_translate_audio("vocals.wav", target_language='hi')
# Specify the file path
output_file_path = "vocal_text.txt"

# Export the value to a text file
with open(output_file_path, "w", encoding="utf-8") as output_file:
    output_file.write(vocals_text)

print(f"Exported vocals_text to {output_file_path}")




Exported vocals_text to vocal_text.txt


#Easy Speaking
○ Read contents of vocal_text.txt <br>
○ Pass thru Coqui and get output.wav

♥ ♥ ♥

In [16]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)
# Read the contents of vocal_text.txt
with open("vocal_text.txt", "r", encoding="utf-8") as file:
    vocals_text = file.read()

# Run TTS
# ❗ Since this model is multi-lingual voice cloning model, we must set the target speaker_wav and language
# Text to speech list of amplitude values as output
wav = tts.tts(text=vocals_text, speaker_wav="/content/vocals.wav", language="hi")
# Text to speech to a file
tts.tts_to_file(text=vocals_text, speaker_wav="/content/vocals.wav", language="hi", file_path="output.wav")



No API token found for 🐸Coqui Studio voices - https://coqui.ai 
Visit 🔗https://app.coqui.ai/account to get one.
Set it as an environment variable `export COQUI_STUDIO_TOKEN=<token>`

['tts_models/multilingual/multi-dataset/xtts_v2', 'tts_models/multilingual/multi-dataset/xtts_v1.1', 'tts_models/multilingual/multi-dataset/your_tts', 'tts_models/multilingual/multi-dataset/bark', 'tts_models/bg/cv/vits', 'tts_models/cs/cv/vits', 'tts_models/da/cv/vits', 'tts_models/et/cv/vits', 'tts_models/ga/cv/vits', 'tts_models/en/ek1/tacotron2', 'tts_models/en/ljspeech/tacotron2-DDC', 'tts_models/en/ljspeech/tacotron2-DDC_ph', 'tts_models/en/ljspeech/glow-tts', 'tts_models/en/ljspeech/speedy-speech', 'tts_models/en/ljspeech/tacotron2-DCA', 'tts_models/en/ljspeech/vits', 'tts_models/en/ljspeech/vits--neon', 'tts_models/en/ljspeech/fast_pitch', 'tts_models/en/ljspeech/overflow', 'tts_models/en/ljspeech/neural_hmm', 'tts_models/en/vctk/vits', 'tts_models/en/vctk/fast_pitch', 'tts_models/en/sam/tacotron-D

100%|█████████▉| 1.86G/1.87G [00:27<00:00, 81.1MiB/s]
100%|██████████| 1.87G/1.87G [00:27<00:00, 67.5MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 12.2kiB/s]

100%|██████████| 361k/361k [00:00<00:00, 837kiB/s]


 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts
 > Text splitted to sentences.
['जब मैं छोटा था तब मेरे पिता एक सेना थे इसलिए हमारे पास बहुत सीमित बजट था इसलिए मैंने अपने जूते बदल दिए और उन्हीं जूतों के साथ खेला इसलिए जब मैं बड़ा हो रहा था तो मैं हर बार पिछले 9 साल से खेलने के लिए जूते नहीं खरीद सकता था हर बार जब मैंने कहा कि आज मैं इसे खरीद सकता हूं']
[!] Warning: The text length exceeds the character limit of 250 for language 'hi', this might cause truncated audio.
 > Processing time: 23.470921277999878
 > Real-time factor: 0.9197265599318599
 > Text splitted to sentences.
['जब मैं छोटा था तब मेरे पिता एक सेना थे इसलिए हमारे पास बहुत सीमित बजट था इसलिए मैंने अपने जूते बदल दिए और उन्हीं जूतों के साथ खेला इसलिए जब मैं बड़ा हो रहा था तो मैं हर बार पिछले 9 साल से खेलने के लिए जूते नहीं खरीद सकता था हर बार जब मैंने कहा कि आज मैं इसे खरीद सकता हूं']
[!] Warning: The text length exceeds the character limit of 250 for language 'hi', this might

'output.wav'

# Your work is done, Thanks for using ♥ ♥ ♥

In [17]:
#Final Stretch
import subprocess

# Input video file path
input_video_path = "only_video.mp4"

# Input audio file path
input_audio_path = "output.wav"

# Output merged video file path
output_merged_path = "Final_output.mp4"

# Run ffmpeg command to merge video and audio
subprocess.run(["ffmpeg", "-i", input_video_path, "-i", input_audio_path, "-c:v", "copy", "-c:a", "aac", "-strict", "experimental", "-map", "0:v:0", "-map", "1:a:0", output_merged_path])



video       -------------- (English)
spanish Vid ------------------------()



CompletedProcess(args=['ffmpeg', '-i', 'only_video.mp4', '-i', 'output.wav', '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental', '-map', '0:v:0', '-map', '1:a:0', 'Final_output.mp4'], returncode=0)